IMPORTO LIBRERÍAS QUE VOY A UTILIZAR

In [42]:
# Tratamiento de datos
import numpy as np
import pandas as pd

# Gráficos
import matplotlib.pyplot as plt

# Preprocesado y modelado
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix
from sklearn.metrics import plot_confusion_matrix
from sklearn.metrics import classification_report
from sklearn.model_selection import train_test_split
from sklearn.model_selection import RepeatedKFold
from sklearn.model_selection import ParameterGrid
from sklearn.model_selection import train_test_split
from sklearn.metrics import recall_score
from sklearn import preprocessing

INGESTO LOS DATOS: en este caso contamos con 2 archivos .csv que contienen información sobre la estadía de pacientes en instituciones hospitalarias. Se pretende generar un modelo de clasificación que permita predecir si un paciente tendrá una estadía larga (más de 8 días) o corta (8 días o menos) en función de las variables disponibles.

In [43]:
train ='./datasets/hospitalizaciones_train.csv'
df_train= pd.read_csv(train)

test ='./datasets/hospitalizaciones_test.csv'
df_test= pd.read_csv(test)



Empezamos con nuestro Análisis exploratorio de datos!

ACLARACIÓN: si bien vamos a utilizar df_train para entrenar nuestro modelo y df_test para buscar las predicciones, los cambios aplicados como parte del EDA se realizarán en ambos dataframes.

Utilizo .head() .info() .duplicate() para obtener un panoráma general de mi dataframe, ver los tipos de variables, si hay filas duplicadas y si existen valores nulos.

In [ ]:
df_train.head()

In [ ]:
df_train.info()

In [ ]:
df_train.duplicated().sum()

Como mi modelo busca responder un problemática de clasificación (estadía larga, estadía corta), lo primero que tenemos que hacer es transformar nuestra variable numérica 'Stay (in days)' en una variable Binaria: 0 (≤ 8 días = estadía corta) y 1 (> 8 días = estadía larga), esta transformación va a estar reflejada en una nueva columna llamada 'Estadia'.

In [44]:
df_train["Estadia"] = np.where(df_train["Stay (in days)"]>8,1,0)


Ya tenemos nuestra nueva columna, ahora pasamos a eliminar la columna 'Stay (in days)', también elimino otras columnas que considero no tienen correlacion con la predicción que queremos hacer.
-Ward_Facility_Code: Código de la habitación del paciente.
-doctor_name: Nombre de el/la doctor/a a cargo del paciente.
-staff_available: Cantidad de personal disponible al momento del ingreso del paciente.
-patientid: Identificador del paciente.
-Visitors with Patient: Cantidad de visitantes registrados para el paciente.
-Admission_Deposit: Pago realizado a nombre del paciente, con el fin de cubrir los costos iniciales de internación.

In [45]:
df_train = df_train.drop(columns=['Stay (in days)','Ward_Facility_Code','doctor_name','staff_available','patientid','Visitors with Patient','Admission_Deposit','Available Extra Rooms in Hospital'])

In [46]:
df_test = df_test.drop(columns=['Ward_Facility_Code','doctor_name','staff_available','patientid','Visitors with Patient','Admission_Deposit','Available Extra Rooms in Hospital'])

In [ ]:
df_train.head()

Para poder comenzar a entrenar el modelo, voy a necesitar transformar las variables categóricas a variables numéricas, para ello voy a utilizar .LabelEncoder()

In [47]:
cat = preprocessing.LabelEncoder()

cat.fit(df_train.Department)
df_train.Department = cat.transform(df_train.Department)

cat.fit(df_train.Age)
df_train.Age = cat.transform(df_train.Age)

cat.fit(df_train.gender)
df_train.gender = cat.transform(df_train.gender)

cat.fit(df_train['Type of Admission'])
df_train['Type of Admission'] = cat.transform(df_train['Type of Admission'])

cat.fit(df_train['Severity of Illness'])
df_train['Severity of Illness'] = cat.transform(df_train['Severity of Illness'])

cat.fit(df_train.health_conditions)
df_train.health_conditions = cat.transform(df_train.health_conditions)

cat.fit(df_train.Insurance)
df_train.Insurance = cat.transform(df_train.Insurance)

In [48]:
cat = preprocessing.LabelEncoder()

cat.fit(df_test.Department)
df_test.Department = cat.transform(df_test.Department)

cat.fit(df_test.Age)
df_test.Age = cat.transform(df_test.Age)

cat.fit(df_test.gender)
df_test.gender = cat.transform(df_test.gender)

cat.fit(df_test['Type of Admission'])
df_test['Type of Admission'] = cat.transform(df_test['Type of Admission'])

cat.fit(df_test['Severity of Illness'])
df_test['Severity of Illness'] = cat.transform(df_test['Severity of Illness'])

cat.fit(df_test.health_conditions)
df_test.health_conditions = cat.transform(df_test.health_conditions)

cat.fit(df_test.Insurance)
df_test.Insurance = cat.transform(df_test.Insurance)

In [49]:
df_train.head()

,Department,Age,gender,Type of Admission,Severity of Illness,health_conditions,Insurance,Estadia
0,2,4,0,1,0,1,1,0
1,2,3,0,1,1,2,0,1
2,2,2,0,1,0,1,1,0
3,2,3,0,2,2,4,0,0
4,1,7,1,1,2,1,0,1


In [57]:
x= df_train.drop(['Estadia'], axis= 1)
y= df_train['Estadia']

In [58]:
X_train, X_test, y_train, y_test = train_test_split(x, y, test_size=0.30, random_state=42, stratify=y)

In [125]:
# Grid de hiperparámetros evaluados
# ==============================================================================
param_grid = ParameterGrid(
                {'n_estimators': [150],
                 'max_features': [5, 7, 9],
                 'max_depth'   : [None, 3, 10, 20, 30, 40],
                 'criterion'   : ['gini', 'entropy']
                }
            )

# Loop para ajustar un modelo con cada combinación de hiperparámetros
# ==============================================================================
resultados = {'params': [], 'oob_accuracy': []}

for params in param_grid:
    
    model = RandomForestClassifier(
                oob_score    = True,
                n_jobs       = -1,
                random_state = 123,
                ** params
             )
    
    model.fit(X_train, y_train)
    
    resultados['params'].append(params)
    resultados['oob_accuracy'].append(model.oob_score_)

# Resultados
# ==============================================================================
resultados = pd.DataFrame(resultados)
resultados = pd.concat([resultados, resultados['params'].apply(pd.Series)], axis=1)
resultados = resultados.sort_values('oob_accuracy', ascending=False)
resultados = resultados.drop(columns = 'params')
resultados.head(4)

,oob_accuracy,criterion,max_depth,max_features,n_estimators
7,0.698098,gini,10.0,7,150
24,0.697756,entropy,10.0,5,150
6,0.697710,gini,10.0,5,150
25,0.697235,entropy,10.0,7,150


In [59]:
model = RandomForestClassifier(
                oob_score    = True,
                n_jobs       = -1,
             )

In [60]:
model.fit(X_train, y_train)

RandomForestClassifier(n_jobs=-1, oob_score=True)

In [61]:
prediction = model.predict(X_test)
cm = confusion_matrix(y_test,prediction)
print("Matriz de confusión:")
print(cm)

Matriz de confusión:
[[11976 34399]
 [ 3974 72651]]


In [62]:
print('Exactitud:', accuracy_score(y_test, prediction))

Exactitud: 0.6880243902439024


In [63]:
report = classification_report(y_test, prediction)
print("Reporte de Clasificación:")
print(report)

Reporte de Clasificación:
              precision    recall  f1-score   support

           0       0.75      0.26      0.38     46375
           1       0.68      0.95      0.79     76625

    accuracy                           0.69    123000
   macro avg       0.71      0.60      0.59    123000
weighted avg       0.71      0.69      0.64    123000



In [64]:
tree1 = RandomForestClassifier(criterion= 'gini', max_depth= 10)
tree1.fit(x,y)

RandomForestClassifier(max_depth=10)

In [65]:
prediction1 = pd.DataFrame()
prediction1['pred'] = tree1.predict(df_test)
prediction1.shape

(90000, 1)

In [66]:
prediction1.to_csv(path_or_buf='../MelinaRG.csv' , index=False)